In [1]:
import lightkurve as lk

In [31]:
search = lk.search_lightcurve("Beta Pictoris", mission="TESS")
search

#,mission,year,author,exptime,target_name,distance
,,,,s,,arcsec
0,TESS Sector 04,2018,SPOC,120,270577175,0.0
1,TESS Sector 04,2018,TESS-SPOC,1800,270577175,0.0
2,TESS Sector 04,2018,QLP,1800,270577175,0.0
3,TESS Sector 05,2018,SPOC,120,270577175,0.0
4,TESS Sector 05,2018,TESS-SPOC,1800,270577175,0.0
5,TESS Sector 05,2018,QLP,1800,270577175,0.0
6,TESS Sector 06,2018,SPOC,120,270577175,0.0
7,TESS Sector 06,2018,TESS-SPOC,1800,270577175,0.0
8,TESS Sector 06,2018,QLP,1800,270577175,0.0


In [32]:
from astropy.table import Table, hstack

In [33]:
tlc = search[6].download()

In [34]:
tlc.columns

<TableColumns names=('time','flux','flux_err','timecorr','cadenceno','centroid_col','centroid_row','sap_flux','sap_flux_err','sap_bkg','sap_bkg_err','pdcsap_flux','pdcsap_flux_err','quality','psf_centr1','psf_centr1_err','psf_centr2','psf_centr2_err','mom_centr1','mom_centr1_err','mom_centr2','mom_centr2_err','pos_corr1','pos_corr2')>

In [47]:
table = hstack([tlc.time,tlc.flux,tlc.quality])
table

time,flux,quality
,electron / s,
object,float32,int32
1468.273598025441,5.5032510e+06,0
1468.2749869153856,5.5023390e+06,0
1468.276375805329,5.5035595e+06,0
1468.2777646952732,5.5049100e+06,0
1468.2791535852177,5.5062295e+06,0
1468.2805424751618,5.5089420e+06,0
1468.281931365106,5.5126150e+06,0
1468.2833202550498,5.5148900e+06,0


### Automated Search

In [11]:
cd ..

/Users/azib/Documents/PhD/Exocomets/automated_exocomet_hunt


In [42]:
from analysis_tools_cython import *

In [43]:
timestep = calculate_timestep(table)
""""The default assumption is a 30-minute cadence."""
factor = ((1/48)/timestep)
 

t, flux, quality, real = clean_data(table)
N = len(t)
ones = np.ones(N)

flux = normalise_flux(flux)

# filteredflux = fourier_filter(flux, 8) # returns smooth lc
A_mag = np.abs(np.fft.rfft(flux))
# periodicnoise = flux - filteredflux


sigma = flux.std()

flux_ls = np.copy(flux)
lombscargle_filter(t, flux_ls, real, 0.05)  # happens in-place. 0.05 is minimum score
periodicnoise_ls = flux - flux_ls
flux_ls = flux_ls * real

# T1 = test_statistic_array(filteredflux, 60)
T = test_statistic_array(flux_ls, 60 * factor)
data = nonzero(T)

# Find minimum test statistic value, and its location.
m, n = np.unravel_index(T.argmin(), T.shape)
minT = T[m, n]
minT_time = t[n]
minT_duration = m * timestep
print("Timestep: ", round(timestep,2))
print("Maximum transit chance:")
print("   Time =", round(minT_time, 2), "days.")
print("   Duration =", round(minT_duration, 2), "days.")
print("   T =", round(minT, 1))
print("   T/sigma =", round(minT / data.std(), 1))

trans_start = n - math.floor((m - 1) / 2)
trans_end = trans_start + m
print("Transit depth =", round(flux[trans_start:trans_end].mean(), 6))

# Transit shape calculation
if n - 3 * m >= 0 and n + 3 * m < N:
    t2 = t[n - 3 * m : n + 3 * m]
    x2 = flux_ls[n - 3 * m : n + 3 * m]
    q2 = quality[n - 3 * m : n + 3 * m]
    background = (sum(x2[: 1 * m]) + sum(x2[5 * m :])) / (2 * m)
    x2 -= background
    paramsgauss = single_gaussian_curve_fit(t2, -x2)
    y2 = -gauss(t2, *paramsgauss)
    paramscomet = comet_curve_fit(t2, -x2)
    w2 = -comet_curve(t2, *paramscomet)

    scores = [score_fit(x2, fit) for fit in [y2, w2]]
    print(scores)
    print("Asym score:", round(scores[0] / scores[1], 4))

    qual_flags = reduce(lambda a, b: a or b, q2)
    print("Quality flags:", qual_flags)

# Classify events
asym, _, _ = calc_shape(m, n, t, flux)
print(classify(m, n, real, asym))

# Skip plotting if no graphical output set
if args.n:
    sys.exit()

# plt.xkcd()
fig1, axarr = plt.subplots(4)
axarr[0].plot(A_mag)  # fourier plot
axarr[0].title.set_text("Fourier plot")
axarr[1].plot(t, flux + ones, t, periodicnoise_ls + ones)  #
axarr[2].plot(t, flux_ls + ones)  # lomb-scargle plot
axarr[2].title.set_text("Lomb-Scargle plot")
cax = axarr[3].imshow(T)
axarr[3].set_aspect("auto")
fig1.colorbar(cax)

# params = double_gaussian_curve_fit(T)
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
# T_test_nonzero = np.array(data)
# _,bins,_ = ax2.hist(T_test_nonzero,bins=100,log=True)
# y = np.maximum(bimodal(bins,*params),10)
# ax2.plot(bins,y)
try:
    ax2.plot(t2, x2, t2, y2, t2, w2)
except:
    pass

plt.show()

TypeError: type Quantity doesn't define __round__ method